# Pull data from World Bank WDI and save to local CSV

In [1]:
"""
Pull data from World Bank WDI and save to local CSV

Dependencies:
    pip install wbgapi pandas
"""

import wbgapi as wb
import pandas as pd
import os

def fetch_and_save_wdi(
    indicators: list,
    countries: list = None,
    years: list = None,
    output_dir: str = "data/raw",
    output_filename: str = "wdi_data.csv"
):
    """
    Fetch World Bank WDI data and save as CSV.

    Parameters:
        indicators: List of indicator codes, e.g. ["NY.GDP.PCAP.KD", "SL.EMP.TOTL.SP.ZS", "NY.GDP.MKTP.KD.ZG"]
        countries: List of country ISO-2/ISO-3 codes, default None means all available countries
        years: List of year range, e.g. list(range(2000, 2024))
        output_dir: Local save directory
        output_filename: Output file name
    """
    os.makedirs(output_dir, exist_ok=True)

    if countries is None:
        countries = wb.economy.list()  # Get codes for all economies

    # If years not specified, use 1960–latest
    if years is None:
        years = list(range(1960, pd.Timestamp.now().year + 1))

    # Fetch data: DataFrame with row index (economy, year), columns as indicators
    df = wb.data.DataFrame(
        indicators,
        economy=countries,
        time=years,
        labels=True
    )

    # Reset index, convert economy and time to regular columns
    df = df.reset_index().rename(columns={"economy": "country", "time": "year"})

    # Save as CSV (no index column)
    output_path = os.path.join(output_dir, output_filename)
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"Saved WDI data to {output_path}")

if __name__ == "__main__":
    indicators = [
        "NY.GDP.PCAP.KD",
        "SL.EMP.TOTL.SP.ZS",
        "NY.GDP.MKTP.KD.ZG"
    ]

    # ["USA","CHN","IND"]
    countries = ["USA", "CHN", "IND"]

    # Specify year range
    years = list(range(2000, 2024))

    # Execute fetch and save
    fetch_and_save_wdi(
        indicators=indicators,
        countries=countries,
        years=years,
        output_dir="../data/raw",
        output_filename="economic_dev_2000_2023.csv"
    )

Saved WDI data to ../data/raw/economic_dev_2000_2023.csv


# Data cleaning

In [2]:
"""
Use DuckDB in Python to perform SQL cleaning/unpivoting of WDI wide table, and export long-format CSV.
Dependencies:
    pip install duckdb pandas
"""

import duckdb
import pandas as pd
import os

RAW_CSV        = "../data/raw/economic_dev_2000_2023.csv"
OUTPUT_CSV     = "../data/clean/wdi_long_clean.csv"
DB_FILE        = "../data/tmp/wdi.duckdb"

os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
os.makedirs(os.path.dirname(DB_FILE), exist_ok=True)

# 1) Connect to DuckDB (file storage), or create in-memory database with ":memory:"
con = duckdb.connect(database=DB_FILE, read_only=False)

# 2) Read CSV into a DuckDB table raw_widi, rename to avoid case conflicts in SELECT
con.execute(f"""
CREATE OR REPLACE TABLE raw_wdi AS
SELECT
    country        AS country_code,
    series         AS indicator_code,
    "Country"      AS country_name,
    "Series"       AS indicator_name,
    YR2000, YR2001, YR2002, YR2003, YR2004, YR2005,
    YR2006, YR2007, YR2008, YR2009, YR2010, YR2011,
    YR2012, YR2013, YR2014, YR2015, YR2016, YR2017,
    YR2018, YR2019, YR2020, YR2021, YR2022, YR2023
FROM read_csv_auto('{RAW_CSV}');
""")

# 3) Use SQL UNPIVOT to transform wide table to long table, filter out NULLs
con.execute("""
CREATE OR REPLACE TABLE clean_wdi AS
SELECT
    country_code,
    indicator_code,
    country_name,
    indicator_name,
    CAST(REPLACE(col, 'YR', '') AS INTEGER) AS year,
    value
FROM raw_wdi
UNPIVOT (
    value FOR col IN (
        YR2000, YR2001, YR2002, YR2003, YR2004, YR2005,
        YR2006, YR2007, YR2008, YR2009, YR2010, YR2011,
        YR2012, YR2013, YR2014, YR2015, YR2016, YR2017,
        YR2018, YR2019, YR2020, YR2021, YR2022, YR2023
    )
)
WHERE value IS NOT NULL
ORDER BY country_code, indicator_code, year;
""")

# 4) Export cleaned long table to local CSV
df_clean = con.execute("SELECT * FROM clean_wdi").df()
df_clean.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

con.close()
print("Clean data saved to", OUTPUT_CSV)

Clean data saved to ../data/clean/wdi_long_clean.csv


# Exploratory analysis

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Configuration
INPUT_CSV = "../data/clean/wdi_long_clean.csv"
FIGURES_DIR = "../figures/exploratory"
os.makedirs(FIGURES_DIR, exist_ok=True)

# Load data
df = pd.read_csv(INPUT_CSV)

# Descriptive statistics table (as before)
desc = df.groupby(['indicator_code','country_code'])['value'].describe().round(2)
desc.to_csv(os.path.join(FIGURES_DIR, "descriptive_stats.csv"))
pivot_mean = df.groupby(['country_code','indicator_code'])['value'].mean().unstack().round(2)
pivot_mean.to_csv(os.path.join(FIGURES_DIR, "mean_pivot.csv"))

# 1. Trend line chart: Increase size, mark, annotate last value
for indicator in df['indicator_code'].unique():
    sub = df[df['indicator_code']==indicator]
    plt.figure(figsize=(10, 6))
    for country in sub['country_code'].unique():
        csub = sub[sub['country_code']==country]
        plt.plot(csub['year'], csub['value'], marker='o', linewidth=2, label=country)
        # Annotate last point
        last = csub.iloc[-1]
        plt.text(last['year'], last['value'], f"{last['value']:.1f}", va='bottom', ha='right')
    plt.title(f"Trend of {indicator}", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Value", fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(title="Country")
    plt.grid(linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"trend_{indicator}.png"))
    plt.close()

# 2. Bar comparison for 2023: Show value on bars
latest = df[df['year']==df['year'].max()]
for indicator in latest['indicator_code'].unique():
    sub = latest[latest['indicator_code']==indicator]
    plt.figure(figsize=(8, 5))
    bars = plt.bar(sub['country_code'], sub['value'])
    plt.title(f"{indicator} in {int(df['year'].max())}", fontsize=14)
    plt.xlabel("Country", fontsize=12)
    plt.ylabel("Value", fontsize=12)
    # Annotate value at bar top
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, h, f"{h:.1f}", ha='center', va='bottom')
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"bar_{indicator}_2023.png"))
    plt.close()

# 3. Correlation heatmap: Use pcolormesh and add annotations
for country in df['country_code'].unique():
    sub = df[df['country_code']==country]
    wide = sub.pivot(index='year', columns='indicator_code', values='value')
    corr = wide.corr()
    plt.figure(figsize=(6, 5))
    mesh = plt.pcolormesh(corr.values, edgecolors='k', linewidth=0.5)
    plt.xticks(range(len(corr)), corr.columns, rotation=45)
    plt.yticks(range(len(corr)), corr.index)
    plt.title(f"Indicator Correlation ({country})", fontsize=14)
    # Annotate each cell
    for i in range(corr.shape[0]):
        for j in range(corr.shape[1]):
            plt.text(j + 0.5, i + 0.5, f"{corr.iloc[i,j]:.2f}", ha='center', va='center')
    plt.colorbar(mesh)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"corr_{country}.png"))
    plt.close()

# 4. Histogram: Overall distribution of indicators
for indicator in df['indicator_code'].unique():
    sub = df[df['indicator_code']==indicator]
    plt.figure(figsize=(8, 5))
    plt.hist(sub['value'], bins=15)
    plt.title(f"Distribution of {indicator}", fontsize=14)
    plt.xlabel("Value", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    plt.grid(axis='y', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"hist_{indicator}.png"))
    plt.close()

# 5. Boxplot: Compare indicator distribution by country
for indicator in df['indicator_code'].unique():
    sub = df[df['indicator_code']==indicator]
    plt.figure(figsize=(8, 5))
    data = [sub[sub['country_code']==c]['value'] for c in sub['country_code'].unique()]
    plt.boxplot(data, labels=sub['country_code'].unique())
    plt.title(f"Boxplot of {indicator} by Country", fontsize=14)
    plt.xlabel("Country", fontsize=12)
    plt.ylabel("Value", fontsize=12)
    plt.grid(axis='y', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"box_{indicator}.png"))
    plt.close()

# 6. Trend smoothing: Moving average (window=3)
for indicator in df['indicator_code'].unique():
    sub = df[df['indicator_code']==indicator]
    plt.figure(figsize=(10, 6))
    for country in sub['country_code'].unique():
        csub = sub[sub['country_code']==country].set_index('year')
        smooth = csub['value'].rolling(window=3, center=True).mean()
        plt.plot(smooth.index, smooth.values, linewidth=2, label=country)
    plt.title(f"3-Year Moving Avg of {indicator}", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Smoothed Value", fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(title="Country")
    plt.grid(linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"smooth_{indicator}.png"))
    plt.close()

print("Improved exploratory analysis complete; figures saved to", FIGURES_DIR)

Improved exploratory analysis complete; figures saved to ../figures/exploratory


# Modeling and in-depth mining

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

# Set matplotlib Chinese support
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans', 'Liberation Sans', 'Bitstream Vera Sans', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# Configuration
INPUT_CSV = "../data/clean/wdi_long_clean.csv"
MODELS_DIR = "../models"
FIGURES_DIR = "../figures/modeling"

# Create output directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

# Set random seed
np.random.seed(42)

# Load data
print("Loading data...")
df = pd.read_csv(INPUT_CSV)

# Data overview
print("\nData overview:")
print(f"- Number of rows: {df.shape[0]}")
print(f"- Number of columns: {df.shape[1]}")
print(f"- Country list: {', '.join(df['country_code'].unique())}")
print(f"- Indicator list: {', '.join(df['indicator_code'].unique())}")
print(f"- Year range: {df['year'].min()} - {df['year'].max()}")

# Create wide-format data (each row is a country-year combination, columns are different indicators)
def create_wide_df():
    """Create wide-format data, expanding indicators into columns"""
    # Create a data copy for analysis
    wide_df = df.pivot_table(
        index=['country_code', 'country_name', 'year'],
        columns='indicator_code',
        values='value'
    ).reset_index()
    
    # Rename columns containing dots to avoid errors in formulas
    for col in wide_df.columns:
        if isinstance(col, str) and '.' in col:
            # Replace dots in indicator codes with underscores
            new_col = col.replace('.', '_')
            wide_df = wide_df.rename(columns={col: new_col})
    
    # Create a mapping from indicator code to name
    code_to_name = {}
    for _, row in df.drop_duplicates(['indicator_code', 'indicator_name']).iterrows():
        code = row['indicator_code'].replace('.', '_')
        code_to_name[code] = row['indicator_name']
    
    # Add indicator name columns
    for code, name in code_to_name.items():
        if code in wide_df.columns:
            wide_df[f"{code}_name"] = name
    
    return wide_df, code_to_name

wide_df, code_to_name = create_wide_df()
print("\nWide-format data sample:")
print(wide_df.head())

# Get list of indicator codes
indicator_codes = [col for col in wide_df.columns if col in [c.replace('.', '_') for c in df['indicator_code'].unique()]]

# Save wide-format data
wide_df.to_csv(os.path.join(MODELS_DIR, "wide_format_data.csv"), index=False)

#####################################################
# 1. Linear Regression Analysis - Explore Indicator Relationships
#####################################################
print("\n\n1. Performing linear regression analysis...")

def run_linear_regression(X_col, y_col, data, country=None):
    """Run OLS linear regression and return results
    
    Args:
        X_col: Independent variable column name
        y_col: Dependent variable column name
        data: DataFrame
        country: If specified, analyze only data for this country
    
    Returns:
        Regression result object
    """
    if country:
        data = data[data['country_code'] == country]
    
    # Prepare data
    X = data[[X_col]].values
    y = data[y_col].values
    
    # Add constant term
    X = sm.add_constant(X)
    
    # Fit model
    model = sm.OLS(y, X)
    result = model.fit()
    
    return result

# Analyze relationships between all indicator pairs
regression_results = {}

# Create all possible indicator pair combinations
for i, target in enumerate(indicator_codes):
    for j, predictor in enumerate(indicator_codes):
        if i != j:  # Avoid predicting a variable from itself
            key = f"{target} ~ {predictor}"
            result = run_linear_regression(predictor, target, wide_df)
            
            # Get coefficients (index 1 because index 0 corresponds to constant term)
            coef = result.params[1]
            p_val = result.pvalues[1]
            
            regression_results[key] = {
                'coefficient': coef,
                'p_value': p_val,
                'r_squared': result.rsquared,
                'result': result
            }
            print(f"Regression {key}: Coefficient = {coef:.4f}, p-value = {p_val:.4f}, R² = {result.rsquared:.4f}")

# Visualize regression lines for each indicator pair
for key, res in regression_results.items():
    target, predictor = key.split(' ~ ')
    result = res['result']
    
    plt.figure(figsize=(10, 6))
    sns.regplot(x=predictor, y=target, data=wide_df, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
    
    # Get indicator names
    target_name = code_to_name.get(target, target)
    predictor_name = code_to_name.get(predictor, predictor)
    
    # Add confidence interval
    x_sorted = wide_df[predictor].sort_values()
    X_sorted = sm.add_constant(x_sorted)
    y_pred = result.predict(X_sorted)
    
    # Calculate prediction confidence interval
    pred_ci = result.get_prediction(X_sorted).conf_int(alpha=0.05)
    
    plt.fill_between(
        x_sorted,
        pred_ci[:, 0],
        pred_ci[:, 1],
        alpha=0.2, color='red'
    )
    
    # Add annotations
    plt.title(f"Relationship between {target_name} and {predictor_name}", fontsize=14)
    plt.xlabel(predictor_name, fontsize=12)
    plt.ylabel(target_name, fontsize=12)
    
    # Add regression equation and R²
    eq_text = f"y = {result.params[1]:.4f}x + {result.params[0]:.4f}"
    r2_text = f"R² = {result.rsquared:.4f}"
    p_text = f"p-value = {result.pvalues[1]:.4f}"
    
    plt.annotate(eq_text + '\n' + r2_text + '\n' + p_text, 
                xy=(0.05, 0.95), xycoords='axes fraction',
                bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.8),
                va='top')
    
    plt.grid(linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"regression_{target}_vs_{predictor}.png"))
    plt.close()

# Save main regression results
pd.DataFrame([
    {
        'target': k.split(' ~ ')[0],
        'predictor': k.split(' ~ ')[1],
        'coefficient': v['coefficient'],
        'p_value': v['p_value'],
        'r_squared': v['r_squared']
    }
    for k, v in regression_results.items()
]).to_csv(os.path.join(MODELS_DIR, "regression_results.csv"), index=False)

#####################################################
# 2. Panel Data Analysis - Control for Country and Time Fixed Effects
#####################################################
print("\n\n2. Performing panel data analysis...")

# Create data for panel analysis (add country and year dummy variables)
panel_df = wide_df.copy()

# Convert countries to dummy variables
panel_df = pd.get_dummies(panel_df, columns=['country_code'], drop_first=True)

# For each indicator, perform panel regression analysis
panel_results = {}

for target in indicator_codes:
    predictors = [ind for ind in indicator_codes if ind != target]
    
    # Prepare dataset
    y = panel_df[target]
    
    # Basic model (without fixed effects)
    X_basic = panel_df[predictors].copy()
    X_basic = sm.add_constant(X_basic)
    basic_model = sm.OLS(y, X_basic)
    basic_result = basic_model.fit()
    
    # Fixed effects model (add country dummy variables)
    country_dummies = [col for col in panel_df.columns if col.startswith('country_code_')]
    X_fe = panel_df[predictors + country_dummies].copy()
    for col in X_fe.columns:
        X_fe[col] = pd.to_numeric(X_fe[col], errors='coerce')
    y = pd.to_numeric(panel_df[target], errors='coerce')

    df_model = pd.DataFrame({'y': y}).join(X_fe)
    df_model = df_model.dropna()
    y_clean = df_model['y']
    X_clean = df_model.drop(columns=['y'])

    X_clean = sm.add_constant(X_clean)
    X_clean = X_clean.astype(float)
    y_clean = y_clean.astype(float)

    fe_result = sm.OLS(y_clean, X_clean).fit()
    
    # Time trend model (add year as a linear trend)
    X_trend = panel_df[predictors + country_dummies + ['year']].copy()
    X_trend = sm.add_constant(X_trend)

    X_trend = X_trend.astype(float)
    y_trend = pd.to_numeric(panel_df[target], errors='coerce').astype(float)

    df_trend = pd.concat([y_trend.rename(target), X_trend], axis=1).dropna()
    y_trend_clean = df_trend[target]
    X_trend_clean = df_trend.drop(columns=[target])

    trend_result = sm.OLS(y_trend_clean, X_trend_clean).fit()
    
    panel_results[target] = {
        'basic': basic_result,
        'fixed_effects': fe_result,
        'time_trend': trend_result
    }
    
    # Print results summary
    target_name = code_to_name.get(target, target)
    print(f"\nTarget variable: {target} ({target_name})")
    print(f"Basic model R²: {basic_result.rsquared:.4f}")
    print(f"Fixed effects model R²: {fe_result.rsquared:.4f}")
    print(f"Time trend model R²: {trend_result.rsquared:.4f}")

# Save panel regression results
for target, models in panel_results.items():
    target_name = code_to_name.get(target, target)
    
    # Create a comparison table
    coef_table = pd.DataFrame()
    
    # For each predictor, extract coefficients and p-values
    predictors = [ind for ind in indicator_codes if ind != target]
    
    for predictor in predictors:
        predictor_name = code_to_name.get(predictor, predictor)
        
        # Extract coefficients and p-values by predictor name
        coef_basic = models['basic'].params.get(predictor, np.nan)
        p_basic    = models['basic'].pvalues.get(predictor, np.nan)

        coef_fe    = models['fixed_effects'].params.get(predictor, np.nan)
        p_fe       = models['fixed_effects'].pvalues.get(predictor, np.nan)

        coef_trend = models['time_trend'].params.get(predictor, np.nan)
        p_trend    = models['time_trend'].pvalues.get(predictor, np.nan)
        
        # Add to table
        row = {
            'Predictor': predictor_name,
            'Basic Model_Coefficient': coef_basic,
            'Basic Model_p-value': p_basic,
            'Fixed Effects_Coefficient': coef_fe,
            'Fixed Effects_p-value': p_fe,
            'Time Trend_Coefficient': coef_trend,
            'Time Trend_p-value': p_trend
        }
        coef_table = pd.concat([coef_table, pd.DataFrame([row])], ignore_index=True)
    
    # Add model information
    info_row = {
        'Predictor': 'Model Information',
        'Basic Model_Coefficient': f"R²: {models['basic'].rsquared:.4f}",
        'Basic Model_p-value': f"Adj R²: {models['basic'].rsquared_adj:.4f}",
        'Fixed Effects_Coefficient': f"R²: {models['fixed_effects'].rsquared:.4f}",
        'Fixed Effects_p-value': f"Adj R²: {models['fixed_effects'].rsquared_adj:.4f}",
        'Time Trend_Coefficient': f"R²: {models['time_trend'].rsquared:.4f}",
        'Time Trend_p-value': f"Adj R²: {models['time_trend'].rsquared_adj:.4f}"
    }
    coef_table = pd.concat([coef_table, pd.DataFrame([info_row])], ignore_index=True)
    
    # Save table
    coef_table.to_csv(os.path.join(MODELS_DIR, f"panel_results_{target}.csv"), index=False)
    
    # Visualize model comparison
    plt.figure(figsize=(12, 6))
    
    # Plot original data
    for country in df['country_code'].unique():
        country_data = wide_df[wide_df['country_code'] == country]
        plt.scatter(country_data['year'], country_data[target], alpha=0.7, label=country if len(df['country_code'].unique()) <= 5 else None)
    
    # Plot different model predictions
    sorted_data = panel_df.sort_values(['year'])
    
    # Predict basic model
    y_pred_basic = models['basic'].predict(X_basic)
    
    # Plot basic model prediction
    plt.plot(sorted_data['year'], y_pred_basic, 'r-', linewidth=2, label='Basic Model')
    
    plt.title(f"{target_name} - Panel Data Models", fontsize=14)
    plt.xlabel("Year", fontsize=12)
    plt.ylabel(target_name, fontsize=12)
    plt.legend(title="Model/Country")
    plt.grid(linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_DIR, f"panel_models_{target}.png"))
    plt.close()